In [1]:
import os
import pandas as pd
import json
from dotenv import find_dotenv, load_dotenv
from helper.misc import canUpdateData, loadMALJSON_DF, combineMALDFs
from helper.download_data import myMALData
from dateutil.relativedelta import relativedelta
from IPython.display import display

In [2]:
dotenv_file = find_dotenv()
load_dotenv(dotenv_file)

print('Checking MAL Manga Data Import date...')
if canUpdateData(dotenv_file, 'nextMALImportDate_Manga', '%Y-%m-%d', 2):
    clientId = os.getenv('X-MAL-CLIENT-ID')
    mangaList = os.getenv('myMangaList')
    myListStatus = ['reading', 'completed', 'dropped', 'on_hold', 'plan_to_read']
    fields = ['status, media_type, genres, num_chapters, num_volumes, start_date, end_date, list_status']
    filePath = './data/MAL Manga Data/'
    myMALData(clientId, mangaList, fields, filePath, myListStatus)

Checking MAL Manga Data Import date...
Cannot update data. Next scheduled update is 2022-06-30


In [3]:
with open('./data/MAL Manga Data/reading.json', 'r') as file:
    readingJson = json.load(file)

with open('./data/MAL Manga Data/completed.json', 'r') as file:
    completedJson = json.load(file)

In [4]:
nodeColumns = ['id', 'title', 'status', 'media_type', 'genres', 'num_chapters','start_date', 'end_date']
listStatusColumns = ['status', 'num_chapters_read', 'start_date', 'finish_date']

mangaDF = combineMALDFs(loadMALJSON_DF(readingJson, nodeColumns, listStatusColumns), loadMALJSON_DF(completedJson, nodeColumns, listStatusColumns))
mangaDF['color'] = 'N/A'
mangaDF['label'] = 'N/A'

mangaDF = mangaDF.astype({'id' : 'int64', 
                          'title' : 'string', 
                          'media_type' : 'string', 
                          'status' : 'string', 
                          'start_date' : 'string', 
                          'finish_date' : 'string', 
                          'color' : 'string', 
                          'label' : 'string', 
                          'publish_start_date' : 'string', 
                          'publish_end_date' : 'string', 
                          'num_chapters' : 'int64', 
                          'num_chapters_read' : 'int64'})

mangaDF

,id,title,publish_status,media_type,genres,num_chapters,publish_start_date,publish_end_date,status,num_chapters_read,start_date,finish_date,color,label
0,118803,Fetiple,finished,manga,"[Comedy, Romance, Slice of Life]",148,2018-08-25,2021-09-25,reading,0,N/A,N/A,N/A,N/A
1,103244,Gekijouban Sword Art Online: Ordinal Scale,finished,manga,"[Action, Adventure, Mystery, Video Game]",20,2017-04-27,2020-04-17,reading,0,N/A,N/A,N/A,N/A
2,130486,Getsuyoubi no Tawawa,currently_publishing,manga,"[Ecchi, Seinen, Slice of Life]",0,2020-11-16,N/A,reading,47,2020-11-15,N/A,N/A,N/A
3,116808,Giji Harem,finished,manga,"[Comedy, Romance, School, Shounen, Slice of Life]",157,2019-01-12,2021-03-12,reading,2,2022-05-01,N/A,N/A,N/A
4,90125,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,currently_publishing,manga,"[Award Winning, Comedy, Romantic Subtext, Scho...",0,2015-05-19,N/A,reading,266,2019-02-18,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,1143,Yuria 100 Shiki,finished,manga,"[Comedy, Ecchi, Seinen]",105,2005-07-22,2010-01-22,completed,105,2019-07-28,2019-09-04,N/A,N/A
450,23212,YuriCam: Yurika no Campus Life,finished,manga,"[Comedy, Ecchi, Erotica, Girls Love, Harem, Se...",45,2010-05-14,2013-01-11,completed,45,2020-10-04,2020-10-17,N/A,N/A
451,1154,Zero no Tsukaima,finished,manga,"[Comedy, Ecchi, Harem, Isekai, Romance, School...",45,2006-06-27,2009-10-27,completed,45,2014-04-28,2014-04-30,N/A,N/A
452,23280,Zero no Tsukaima Chevalier,finished,manga,"[Action, Comedy, Ecchi, Isekai, Romance, Schoo...",27,2010-01-27,2013-03-27,completed,27,2015-01-11,2015-03-03,N/A,N/A


# 100 Chapters Challenge

Choose a difficulty:
- Easy - 15 Entries
- Med. - 30 Entries
- Hard - 45 Entries

For this challenge the participant has to read a manga* with at least 100 chapters.

*Any type is allowed. MAL only entries! Updated!

For an ongoing manga, you must be caught-up with it, or at least be within a 100 of the latest released translated chapter over the initial first 100 chapters.

*EXAMPLE: One Piece has 855 chapters currently out. You must have read over 755 (855 - 100) to be able to use that entry for the challenge.*

Previously read allowed.

Cannot use the same manga twice.

In [5]:
manga100DF = mangaDF.loc[:, ['id', 'title', 'publish_status', 'num_chapters', 'num_chapters_read', 'status', 'finish_date', 'color', 'label']]

unneededItems = set()
unneededItems.update(manga100DF.index[manga100DF['publish_status'] == 'currently_publishing'].intersection(manga100DF.index[manga100DF['num_chapters_read'] < 100]))
unneededItems.update(manga100DF.index[manga100DF['publish_status'] == 'finished'].intersection(manga100DF.index[manga100DF['num_chapters'] < 100]))
manga100DF.drop(index=unneededItems, columns = ['publish_status'], inplace=True)

manga100DF.loc[manga100DF['status'] == 'completed', ['color', 'label']] = 'green', 'o'
manga100DF.loc[manga100DF['status'] == 'reading', ['color', 'label']] = 'orange', 'x'
manga100DF.sort_values('finish_date', inplace=True, ignore_index=True)
manga100DF['difficulty'] = 'N/A'
for row in range(0, min(91, manga100DF.shape[0])):
    if row <= 14:
        manga100DF.loc[row, 'difficulty'] = 'easy'
    elif row <= 44:
        manga100DF.loc[row, 'difficulty'] = 'medium'
    elif row <= 89:
        manga100DF.loc[row, 'difficulty'] = 'hard'
    else:
        manga100DF.loc[row, 'difficulty'] = 'N/A'

manga100DF

,id,title,num_chapters,num_chapters_read,status,finish_date,color,label,difficulty
0,54,Yu☆Gi☆Oh!,343,343,completed,2008,green,o,easy
1,23,Ranma ½,407,407,completed,2011-06,green,o,easy
2,20,Hikaru no Go,198,198,completed,2011-08,green,o,easy
3,21,Death Note,108,108,completed,2011-12,green,o,easy
4,6140,Darren Shan,114,114,completed,2012,green,o,easy
5,671,To LOVE-Ru,162,162,completed,2012-05,green,o,easy
6,25,Fullmetal Alchemist,116,116,completed,2012-08-01,green,o,easy
7,49,Tennis no Ouji-sama,382,382,completed,2015-05-24,green,o,easy
8,1349,07-Ghost,100,100,completed,2015-06-25,green,o,easy
9,430,Suzuka,171,171,completed,2015-09-30,green,o,easy


# END